In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import time 
import random 
from tqdm import tqdm
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from bertviz import head_view, model_view
random.seed(4248)
np.random.seed(4248)

torch.cuda.manual_seed(4248)

In [ ]:
# load data 
dftrain = pd.read_csv('/content/drive/MyDrive/data/rest_train.csv')
dftest = pd.read_csv('/content/drive/MyDrive/data/rest_test.csv')

In [ ]:
tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-base",do_lower_case=True)
model = DebertaV2ForSequenceClassification.from_pretrained("/content/drive/MyDrive/Fine_tune_BERT")

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
dftest

,text,term,polarity
0,The bread is top notch as well .,bread,1
1,I have to say they have one of the fastest del...,delivery times,1
2,Food is always fresh and hot - ready to eat !,Food,1
3,Did I mention that the coffee is OUTSTANDING ?,coffee,1
4,"Certainly not the best sushi in New York , how...",place,1
...,...,...,...
1115,"Creamy appetizers -- taramasalata , eggplant s...",Creamy appetizers,1
1116,"Creamy appetizers -- taramasalata , eggplant s...",warm pitas,0
1117,"Creamy appetizers -- taramasalata , eggplant s...",taramasalata,1
1118,"Creamy appetizers -- taramasalata , eggplant s...",eggplant salad,1


In [ ]:
idx = 0
sentence = dftest.iloc[idx].text
print('sentence:',sentence)
term = dftest.iloc[idx].term
print('term:',term)
print('True label:',dftest.iloc[idx].polarity + 1)
inputs = tokenizer.encode_plus(sentence,term,return_tensors = 'pt')
attention = model(**inputs)[-1]
print('Predicted label:',model(**inputs).logits.topk(1,1,True,True)[1].item())
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
term_start = inputs['token_type_ids'][0].tolist().index(1)
head_view(attention, tokens, term_start)

sentence: The bread is top notch as well .
term: bread
True label: 2
Predicted label: 2


<IPython.core.display.Javascript object>

In [ ]:
model_view(attention, tokens)

<IPython.core.display.Javascript object>